## The Arrow C Data Interface

Up to now we have focused on explaining the Arrow columnar memory layout and showing you examples of it using `pyarrow` and `nanoarrow`. But this memory layout is meant to be a universal standard for tabular data, not tied to a specific implementation.

While there are specifications to share Arrow data between processes or over the network (e.g. the IPC messages), the **Arrow C Data Interface** is meant to actually zero-copy share the data between different libraries *within the same process* (i.e. actually share the same buffers in memory).

The Arrow C Data Interface defines a set of small C structures

```c
struct ArrowSchema {
  const char* format;
  const char* name;
  const char* metadata;
  int64_t flags;
  int64_t n_children;
  struct ArrowSchema** children;
  struct ArrowSchema* dictionary;

  // Release callback
  void (*release)(struct ArrowSchema*);
  // Opaque producer-specific data
  void* private_data;
};

struct ArrowArray {
  int64_t length;
  int64_t null_count;
  int64_t offset;
  int64_t n_buffers;
  int64_t n_children;
  const void** buffers;
  struct ArrowArray** children;
  struct ArrowArray* dictionary;

  // Release callback
  void (*release)(struct ArrowArray*);
  // Opaque producer-specific data
  void* private_data;
};
```


The C Data Interface passes Arrow data buffers through memory pointers. So, by construction, it allows you to share data from one runtime to another without copying it. Since the data is in standard Arrow in-memory format, its layout is well-defined and unambiguous.

And in the examples up to now, when we created a `nanoarrow.Array` from a `pyarrow` array (or vice versa), we were actually using the Arrow C Data Interface to share the data zero-copy under the hood. 
And you might recognize the structure members from the nanoarrow display we have been using to inspect our data.

### Arrow PyCapsule Interface
